In [ ]:
# default_exp datasets.movielens

# MovieLens Dataset
> Implementation of MovieLens datasets.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
#export
from typing import Any, Iterable, List, Optional, Tuple, Union, Callable
import os

import pandas as pd
import numpy as np

import torch

from recohut.utils.common_utils import *
from recohut.datasets.bases.common import Dataset
from recohut.datasets.bases.interactions import InteractionsDataset, InteractionsDataModule
from recohut.datasets.bases.sequential import SequentialDataset, SequentialDataModule

## ML1m Rating Dataset

### Interactions Dataset

In [ ]:
#export
class ML1mDataset(InteractionsDataset):
    url = "http://files.grouplens.org/datasets/movielens/ml-1m.zip"

    @property
    def raw_file_names(self):
        return 'ratings.dat'

    def download(self):
        path = download_url(self.url, self.raw_dir)
        extract_zip(path, self.raw_dir)
        from shutil import move, rmtree
        move(os.path.join(self.raw_dir, 'ml-1m', self.raw_file_names), self.raw_dir)
        rmtree(os.path.join(self.raw_dir, 'ml-1m'))
        os.unlink(path)

    def load_ratings_df(self):
        df = pd.read_csv(self.raw_paths[0], sep='::', header=None, engine='python')
        df.columns = ['uid', 'sid', 'rating', 'timestamp']
        # drop duplicate user-item pair records, keeping recent ratings only
        df.drop_duplicates(subset=['uid', 'sid'], keep='last', inplace=True)
        return df

In [ ]:
#export
class ML1mDataModule(InteractionsDataModule):
    dataset_cls = ML1mDataset

In [ ]:
class Args:
    def __init__(self):
        self.data_dir = '/content/data'
        self.min_rating = 4
        self.num_negative_samples = 99
        self.min_uc = 5
        self.min_sc = 5
        self.val_p = 0.2
        self.test_p = 0.2
        self.seed = 42
        self.split_type = 'stratified'

args = Args()

In [ ]:
ds = ML1mDataModule(**args.__dict__)
ds.prepare_data()

Processing...


Turning into implicit ratings
Filtering triplets
Densifying index


Done!


### Sequential Dataset

In [ ]:
#export
class ML1mDataset_v2(SequentialDataset):
    url = "http://files.grouplens.org/datasets/movielens/ml-1m.zip"

    @property
    def raw_file_names(self):
        return 'ratings.dat'

    def download(self):
        path = download_url(self.url, self.raw_dir)
        extract_zip(path, self.raw_dir)
        from shutil import move, rmtree
        move(os.path.join(self.raw_dir, 'ml-1m', self.raw_file_names), self.raw_dir)
        rmtree(os.path.join(self.raw_dir, 'ml-1m'))
        os.unlink(path)

    def load_ratings_df(self):
        df = pd.read_csv(self.raw_paths[0], sep='::', header=None, engine='python')
        df.columns = ['uid', 'sid', 'rating', 'timestamp']
        return df

In [ ]:
#export
class ML1mDataModule_v2(SequentialDataModule):
    dataset_cls = ML1mDataset_v2

In [ ]:
class Args:
    def __init__(self):
        self.pad = 0
        self.mask = 1
        self.cap = 0
        self.seed = 42
        self.vocab_size = 10000
        self.channels = 128
        self.dropout = 0.4
        self.learning_rate = 1e-4
        self.history_size = 30
        self.data_dir = '/content/data'
        self.log_dir = '/content/recommender_logs'
        self.model_dir = '/content/recommender_models'
        self.batch_size = 32
        self.shuffle = True
        self.max_epochs = 2
        self.val_epoch = 1
        self.gpus = None
        self.monitor = 'valid_loss'
        self.mode = 'min'

args = Args()

In [ ]:
ds = ML1mDataModule_v2(data_sir=args.data_dir, **args.__dict__)
ds.prepare_data()

In [ ]:
#export
class ML1mDataset_v3(SequentialDataset):
    url = "http://files.grouplens.org/datasets/movielens/ml-1m.zip"

    def __init__(self, data_dir, data_type='train', *args, **kwargs):
        super().__init__(data_dir, data_type, *args, **kwargs)
        if data_type == 'train':
            self.ratings_frame = pd.read_csv(self.processed_paths[0], delimiter=",")
        elif data_type == 'valid':
            self.ratings_frame = pd.read_csv(self.processed_paths[1], delimiter=",")
        elif data_type == 'test':
            self.ratings_frame = pd.read_csv(self.processed_paths[2], delimiter=",")

    @property
    def raw_file_names(self):
        return ['ratings.dat', 'movies.dat', 'users.dat']

    @property
    def processed_file_names(self):
        return ['train.csv', 'valid.csv', 'test.csv']

    def download(self):
        path = download_url(self.url, self.raw_dir)
        extract_zip(path, self.raw_dir)
        from shutil import move, rmtree
        for raw_file_name in self.raw_file_names:
            move(os.path.join(self.raw_dir, 'ml-1m', raw_file_name), self.raw_dir)
        rmtree(os.path.join(self.raw_dir, 'ml-1m'))
        os.unlink(path)

    def load_ratings_df(self):
        df = pd.read_csv(self.raw_paths[0], sep='::', header=None, engine='python')
        df.columns = ['uid', 'sid', 'rating', 'timestamp']
        return df

    def load_movies_df(self):
        df = pd.read_csv(self.raw_paths[1], sep='::', header=None, engine='python')
        df.columns = ["sid", "title", "genres"]
        return df

    def load_users_df(self):
        df = pd.read_csv(self.raw_paths[2], sep='::', header=None, engine='python')
        df.columns = ["uid", "sex", "age_group", "occupation", "zip_code"]
        return df

    def process(self):
        ## movies
        movies = self.load_movies_df()
        movies["year"] = movies["title"].apply(lambda x: x[-5:-1])
        movies.year = pd.Categorical(movies.year)
        movies["year"] = movies.year.cat.codes
        movies["sid"] = movies["sid"].astype(str)

        genres = ["Action","Adventure","Animation","Children's","Comedy","Crime",
                "Documentary","Drama","Fantasy","Film-Noir","Horror","Musical",
                "Mystery","Romance","Sci-Fi","Thriller","War","Western"]

        for genre in genres:
            movies[genre] = movies["genres"].apply(
                lambda values: int(genre in values.split("|"))
            )

        ## users
        users = self.load_users_df()
        users.sex = pd.Categorical(users.sex)
        users["sex"] = users.sex.cat.codes
        users.age_group = pd.Categorical(users.age_group)
        users["age_group"] = users.age_group.cat.codes
        users.occupation = pd.Categorical(users.occupation)
        users["occupation"] = users.occupation.cat.codes
        users.zip_code = pd.Categorical(users.zip_code)
        users["zip_code"] = users.zip_code.cat.codes
        users["uid"] = users["uid"].astype(str)

        # ratings
        ratings = self.load_ratings_df()
        ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
        ratings["sid"] = ratings["sid"].astype(str)
        ratings["uid"] = ratings["uid"].astype(str)

        # Transform the movie ratings data into sequences
        # First, let's sort the the ratings data using the unix_timestamp, 
        # and then group the movie_id values and the rating values by user_id.
        # The output DataFrame will have a record for each user_id, with two 
        # ordered lists (sorted by rating datetime): the movies they have rated,
        # and their ratings of these movies.

        ratings_group = ratings.sort_values(by=["timestamp"]).groupby("uid")

        ratings_data = pd.DataFrame(
            data={
                "uid": list(ratings_group.groups.keys()),
                "sids": list(ratings_group.sid.apply(list)),
                "ratings": list(ratings_group.rating.apply(list)),
                "timestamps": list(ratings_group.timestamp.apply(list)),
            }
        )

        # Now, let's split the movie_ids list into a set of sequences of a fixed 
        # length. We do the same for the ratings. Set the sequence_length variable 
        # to change the length of the input sequence to the model. You can also 
        # change the step_size to control the number of sequences to generate for 
        # each user.
        ratings_data.sids = ratings_data.sids.apply(
            lambda ids: self.create_sequences(ids, self.history_size, self.step_size)
        )
        ratings_data.ratings = ratings_data.ratings.apply(
            lambda ids: self.create_sequences(ids, self.history_size, self.step_size)
        )
        del ratings_data["timestamps"]

        # After that, we process the output to have each sequence in a separate 
        # records in the DataFrame. In addition, we join the user features with 
        # the ratings data.
        ratings_data_movies = ratings_data[["uid", "sids"]].explode(
            "sids", ignore_index=True
        )
        ratings_data_rating = ratings_data[["ratings"]].explode("ratings", ignore_index=True)
        ratings_data_transformed = pd.concat([ratings_data_movies, ratings_data_rating], axis=1)
        ratings_data_transformed = ratings_data_transformed.join(
            users.set_index("uid"), on="uid"
        )
        ratings_data_transformed.sids = ratings_data_transformed.sids.apply(
            lambda x: ",".join(x)
        )
        ratings_data_transformed.ratings = ratings_data_transformed.ratings.apply(
            lambda x: ",".join([str(v) for v in x])
        )
        del ratings_data_transformed["zip_code"]
        ratings_data_transformed.rename(
            columns={"sids": "sequence_sids", "ratings": "sequence_ratings"},
            inplace=True,
        )
        # Finally, we split the data into training and testing splits, with 85% 
        # and 15% of the instances, respectively, and store them to CSV files.
        random_selection = np.random.rand(len(ratings_data_transformed.index)) <= 0.85
        train_data = ratings_data_transformed[random_selection]
        test_data = ratings_data_transformed[~random_selection]

        # save
        train_data.to_csv(self.processed_paths[0], index=False, sep=",")
        test_data.to_csv(self.processed_paths[1], index=False, sep=",")
        test_data.to_csv(self.processed_paths[2], index=False, sep=",")

    def __len__(self):
        return len(self.ratings_frame)

    def __getitem__(self, idx):
        data = self.ratings_frame.iloc[idx]
        user_id = data.uid
        movie_history = eval(data.sequence_sids)
        movie_history_ratings = eval(data.sequence_ratings)
        target_movie_id = movie_history[-1:][0]
        target_movie_rating = movie_history_ratings[-1:][0]
        movie_history = torch.LongTensor(movie_history[:-1])
        movie_history_ratings = torch.LongTensor(movie_history_ratings[:-1])
        sex, age_group, occupation = data.sex, data.age_group, data.occupation
        output = (user_id, movie_history, target_movie_id, movie_history_ratings,
                  target_movie_rating, sex, age_group, occupation)
        return output

In [ ]:
#export
class ML1mDataModule_v3(SequentialDataModule):
    dataset_cls = ML1mDataset_v3

Example

In [ ]:
class Args:
    def __init__(self):
        self.history_size = 8
        self.step_size = 1
        self.data_dir = '/content/data'
        self.log_dir = '/content/recommender_logs'
        self.model_dir = '/content/recommender_models'
        self.batch_size = 32
        self.shuffle = True
        self.max_epochs = 2
        self.val_epoch = 1
        self.gpus = None
        self.monitor = 'valid_loss'
        self.mode = 'min'
        self.window_size = 20

args = Args()

In [ ]:
ds = ML1mDataset_v3(**args.__dict__)
display(ds.ratings_frame.head())
ds.ratings_frame.info()

,uid,sequence_sids,sequence_ratings,sex,age_group,occupation
0,1,"3186,1721,1270,1022,2340,1836,3408,1207","4,4,5,5,3,5,4,4",0,0,10
1,1,"1721,1270,1022,2340,1836,3408,1207,2804","4,5,5,3,5,4,4,5",0,0,10
2,1,"1270,1022,2340,1836,3408,1207,2804,260","5,5,3,5,4,4,5,4",0,0,10
3,1,"2340,1836,3408,1207,2804,260,720,1193","3,5,4,4,5,4,3,5",0,0,10
4,1,"1836,3408,1207,2804,260,720,1193,919","5,4,4,5,4,3,5,4",0,0,10


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 819470 entries, 0 to 819469
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   uid               819470 non-null  int64 
 1   sequence_sids     819470 non-null  object
 2   sequence_ratings  819470 non-null  object
 3   sex               819470 non-null  int64 
 4   age_group         819470 non-null  int64 
 5   occupation        819470 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 37.5+ MB


In [ ]:
ds = ML1mDataModule_v3(**args.__dict__)
ds.prepare_data()

ds.setup()

for batch in ds.train_dataloader():
    print(batch)
    break

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:74: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7."


[tensor([ 214,  802, 2063, 2743, 2106,  675,  388, 4956,   73, 2063, 5585, 4115,
         889, 3044, 5449, 3346, 5253, 1980, 1833, 1714, 2860, 5323, 1764, 1631,
        1790,  482, 5852,  536, 2241, 3620, 2409, 3626]), tensor([[2027,  673,  518,  413, 1894, 1918, 2335],
        [3219,  165, 3697, 2617,  288,  707,   18],
        [3072, 2926, 1307, 3424, 1257, 3210, 1965],
        [3502, 1037,  547,  330, 1391, 2694, 2722],
        [  39, 1057, 3477, 3409,  453, 2005, 3481],
        [2710, 2694, 3298, 2770, 3005, 2355, 2433],
        [ 931, 3386, 1097,  161, 1270, 1225, 1957],
        [2336, 2313, 2291, 1196, 3125, 1183,  443],
        [  70, 3484, 3821, 2399,  240, 3864, 3578],
        [1677,  315,  459, 3064, 3774, 3593, 3751],
        [3783, 3408, 3897, 3911, 3893, 3753, 3649],
        [2105,  170,  533, 1562, 2616,  485,  750],
        [2692,   25, 1247,  265, 1393, 1277, 2020],
        [ 858, 1198,  318,   36, 3421, 1225,  903],
        [1722,  736, 3638, 2617, 2376, 1552,  786],
 

In [ ]:
!tree -h --du -C "{args.data_dir}"

/content/data
├── [ 11M]  processed
│   ├── [2.3M]  data_test_neg.pt
│   ├── [ 95K]  data_test_pos.pt
│   ├── [6.5M]  data_train.pt
│   ├── [2.3M]  data_valid_neg.pt
│   └── [ 95K]  data_valid_pos.pt
└── [ 23M]  raw
    └── [ 23M]  ratings.dat

  35M used in 2 directories, 6 files


## ML100k Dataset

In [ ]:
#export
class ML100kDataset(Dataset):
    url = 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'
    
    def __init__(self, root):
        super().__init__(root)
    
    @property
    def raw_file_names(self) -> str:
        return ['u1.base', 'u1.test', 'u4.test', 'allbut.pl', 'u.item', 
                'ua.test', 'u.occupation', 'u3.test', 'u5.base', 'ub.test', 
                'u2.test', 'u3.base', 'u.genre', 'u.data', 'u4.base', 
                'u5.test', 'u.info', 'README', 'ub.base', 'mku.sh', 'u2.base', 
                'u.user', 'ua.base']

    @property
    def processed_file_names(self) -> str:
        raise NotImplementedError

    def download(self):
        path = download_url(self.url, self.raw_dir)
        extract_zip(path, self.raw_dir)
        from shutil import move, rmtree
        file_names = os.listdir(osp.join(self.raw_dir, 'ml-100k'))   
        for file_name in file_names:
            move(osp.join(self.raw_dir, 'ml-100k', file_name), self.raw_dir)
        rmtree(osp.join(self.raw_dir, 'ml-100k'))
        os.unlink(path)

    def process(self):
        raise NotImplementedError

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d -p recohut

Author: Sparsh A.

Last updated: 2022-01-10 10:04:23

recohut: 0.0.10

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

IPython: 5.5.0
pandas : 1.1.5

